In [1]:
pip install requests pymongo pandas

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\''\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [2]:
# pip install requests pymongo

import os
from datetime import datetime, timezone
import requests
from pymongo import MongoClient, ASCENDING
from datetime import datetime, timezone

In [12]:
# =======================
# Config (live-only)
# =======================
API_KEY = "7bdaae7abc6622e671495ef02578a955"   # your real key
CITY    = "Loni Kalbhor"

MONGO_URI   = os.getenv("MONGO_URI", "mongodb://127.0.0.1:27017")
DB_NAME     = "sales_weatherinfo_db"
WEATHER_COL = "weather"   # existing collection is fine

In [5]:
# =======================
# Helpers
# =======================
def midnight_utc_naive(dt_utc: datetime) -> datetime:
    """Return midnight (00:00:00) in UTC as a naive datetime (Mongo stores as UTC)."""
    dt_utc = dt_utc.astimezone(timezone.utc)
    return datetime(dt_utc.year, dt_utc.month, dt_utc.day, 0, 0, 0, 0)

In [6]:
def ensure_indexes(col):
    # Idempotent; if they already exist, MongoDB keeps them as-is.
    col.create_index([("weather_date", ASCENDING), ("city", ASCENDING)], unique=True)
    col.create_index([("city", ASCENDING), ("weather_date", ASCENDING)])

In [7]:
def upsert_weather(doc: dict, col):
    col.update_one(
        {"weather_date": doc["weather_date"], "city": doc["city"]},
        {"$set": doc},
        upsert=True
    )

In [8]:
# =======================
# Live fetch from OpenWeather (no fallback)
# =======================
def fetch_weather_live(api_key: str, city: str) -> dict:
    url = "https://api.openweathermap.org/data/2.5/weather"
    params = {"q": city, "appid": api_key.strip(), "units": "metric"}

    try:
        r = requests.get(url, params=params, timeout=20)
    except requests.RequestException as e:
        raise RuntimeError(f"Network error calling OpenWeather: {e}")

    # Fail loud with the exact API message (401, 404, 429, etc.)
    if not r.ok:
        try:
            msg = r.json()
        except Exception:
            msg = r.text
        raise RuntimeError(f"OpenWeather error {r.status_code}: {msg} | url={r.url}")

    data = r.json()
    return {
        "weather_date": datetime.now(timezone.utc).replace(hour=0, minute=0, second=0, microsecond=0),  # BSON Date at midnight UTC
        "city": city,
        "temp_c": float(data["main"]["temp"]),
        "humidity": int(data["main"]["humidity"]),
        "description": str(data["weather"][0]["description"]),
        "fetched_at": datetime.now(timezone.utc),  # full timestamp of fetch
        "source": "openweather"
    }

In [9]:
# =======================
# Main
# =======================
def main():
    # Basic sanity check on key
    if not API_KEY or len(API_KEY.strip()) != 32:
        raise ValueError("OPENWEATHER API key looks invalid. Expected 32 chars.")

    client = MongoClient(MONGO_URI)
    col = client[DB_NAME][WEATHER_COL]

    ensure_indexes(col)          # safe even if collection/index exists
    doc = fetch_weather_live(API_KEY, CITY)   # live-only; raises on error
    upsert_weather(doc, col)

    print(f"Stored live weather for {doc['city']} on {doc['weather_date'].date()}.")


In [14]:
if __name__ == "__main__":
    main()

Stored live weather for Loni Kalbhor on 2025-08-26.
